In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression


In [ ]:
data_path = "C:/MLA_Saves_Bk/10002/s4m2_concatenate.npy"

In [ ]:
full_data = np.load(data_path)
predictclass_list = []
trueclass_list = []
for i in range(full_data.shape[0]):
    data = full_data[i,:,:]
    _trueclass_list = data[-3,:]
    true_flags = list((_trueclass_list == 1) | (_trueclass_list == 2))
    trueclass_list.append(int(_trueclass_list[true_flags][0]))
    si = true_flags.index(True)
    ei = true_flags[::-1].index(True)
    #plt.plot(range(len(x[si:-ei])),x[si:-ei])
    stim_data = data[:,si:-ei-1]
    #予測クラスの取得
    predict_counts = stim_data[-2,:]
    predict_count_indexes = []
    prev_pc = -1
    for j in range(len(predict_counts)):
        pc = predict_counts[j]
        if prev_pc != pc:
            prev_pc = pc
            if pc > 0:
                predict_count_indexes.append(j)

    #print(predict_count_indexes)
    def pick_predictclass(pci):
        x = stim_data[-1,pci:pci+100]
        mode = stats.mode(x,keepdims=True)[0]
        if mode == 0:
            for f in x:
                if f != 0:
                    return f
        return mode
    #カウントアップと同時に予測フラグが立つわけではないので最頻値で求める
    _predictclass_list = [int(pick_predictclass(pci)) for pci in predict_count_indexes]
    #plt.plot(range(stim_data[:,pci1:pci1+100].shape[1]),stim_data[-2,pci1:pci1+100])
    #plt.plot(range(stim_data[:,pci1:pci1+100].shape[1]),stim_data[-1,pci1:pci1+100])
    #print(np.std(_predictclass_list))
    predictclass_list.append(_predictclass_list)
#predictclass_list

In [ ]:
predictclass_mode_list = [int(stats.mode(pcl,keepdims=True)[0]) if len(pcl) > 0 else -1 for pcl in predictclass_list]
ans = [trueclass_list[i] == predictclass_mode_list[i] for i in range(len(trueclass_list))]
ans2 = np.array(trueclass_list)[ans] #np.array(predictclass_list) & np.array(ans)
"精度:" , len(ans2)/len(ans)

In [ ]:
std_list = []
left_mean_list = []
left_std_list = []
right_mean_list = []
right_std_list = []
mn_predictclass_list = []
print("真|判別シークエンス|最頻|〇/×|std|mean")
for tl, pl,pml,a in zip(trueclass_list,predictclass_list,predictclass_mode_list,ans):
    std = np.std(pl)
    mean = np.mean(pl)
    #if tl == 2:
    print(tl,pl,pml,"〇" if a else "×",std,mean)
    mn_predictclass_list.append([1 if tl == p else 0 for p in pl])
    std_list.append(std)
    if tl == 1:#1==left
        left_std_list.append(std)
        left_mean_list.append(mean)
    else:
        right_std_list.append(std)
        right_mean_list.append(mean)
    
# 混同行列を作成
cm = confusion_matrix(trueclass_list, predictclass_mode_list)
# 混同行列を表示
print("Confusion Matrix:")
print(cm)
print(f"真が左の平均{np.mean(left_mean_list)} 標準偏差{np.mean(left_std_list)}")
print(f"真が右の平均{np.mean(right_mean_list)} 標準偏差{np.mean(right_std_list)}")

n_conv = 4
b = np.ones(n_conv)/n_conv 
conv = np.convolve(std_list, b, mode="full")
plt.plot(range(len(conv)),conv)
plt.show()


In [ ]:
#判別ポイントごとの平均
_mn_list = [mpl[:len(mn_predictclass_list[0])] for mpl in mn_predictclass_list]
plt.errorbar(range(len(mn_predictclass_list[0])),np.mean(_mn_list,axis=0),yerr=np.std(_mn_list,axis=0),
             capsize=5,ecolor='orange')
plt.title("match 1or0")
plt.show()
print(np.std(_mn_list,axis=0))

def plot_lg(x,y,color):
    # 線形回帰モデル、予測値
    model = LinearRegression()
    model_lin = model.fit(x, y)
    y_lin_fit = model_lin.predict(x)
    plt.plot(x, y_lin_fit, color = color, linewidth=0.5)
    plt.title(title)
def plot_epochs(x,y,title):
    # 回帰分析　線形
    itlist = np.array([(i,t) for i,t in enumerate(trueclass_list) if t == 1]).T
    lx = itlist[0,:].reshape(-1, 1)
    ly = y[lx].reshape(-1, 1)
    lp = plt.scatter(lx,ly,marker="o",label="left hand")
    itlist = np.array([(i,t) for i,t in enumerate(trueclass_list) if t == 2]).T
    rx = itlist[0,:].reshape(-1, 1)
    ry = y[rx].reshape(-1, 1)
    rp = plt.scatter(rx,ry,marker="^",label="right hand")
    plt.legend(loc='upper right',bbox_to_anchor=(1.3, 1))
    plt.draw()
    l_color = lp.get_facecolor()
    r_color = rp.get_facecolor()
    plot_lg(x,y,'#000000')
    plot_lg(lx,ly,l_color)
    plot_lg(rx,ry,r_color)
    plt.show()

title = "Length of time matched (mean)"
x = np.array(range(len(_mn_list))).reshape(-1, 1)
y = np.mean(_mn_list,axis=-1)
plot_epochs(x,y,title)

title = "Length of time matched (std)"
x = np.array(range(len(_mn_list))).reshape(-1, 1)
y = np.std(_mn_list,axis=-1)
plot_epochs(x,y,title)